In [ ]:
nabla = "\u2207"
print(nabla)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
import matplotlib as mpl

# Increase the default limit from 20 MB to, say, 100 MB
mpl.rcParams['animation.embed_limit'] = 100.0

# ------------------------------------------------
# 1) Parameters
# ------------------------------------------------
nx, ny = 128, 128     # Grid size
dx = dy = 1.0         # Spatial resolution
dt = 0.1              # Time step
n_steps = 2000        # Total number of steps
plot_interval = 10    # Save/plot every 10 steps

# Cahn-Hilliard material/physical constants
A      = 1.0          # Coefficient in f(c) = A * c^2 * (1-c)^2
kappa  = 1.0          # Gradient energy coefficient
M      = 1.0          # Mobility

# ------------------------------------------------
# 2) Wavevector grids for FFT
# ------------------------------------------------
kx = 2.0 * np.pi * np.fft.fftfreq(nx, d=dx)
ky = 2.0 * np.pi * np.fft.fftfreq(ny, d=dy)
kx2 = kx**2
ky2 = ky**2
kxx, kyy = np.meshgrid(kx2, ky2, indexing='ij')
k2 = kxx + kyy    # |k|^2
k4 = k2**2        # |k|^4

# ------------------------------------------------
# 3) Initialize composition with strictly zero-mean noise
# ------------------------------------------------
c0 = 0.5           # Desired average composition
noise_amp = 0.005  # Magnitude of the noise (±0.5% around c0)

np.random.seed(0)  # For reproducible results (optional)

# Generate uniform random noise in [-1, +1]
noise = 2.0 * np.random.rand(nx, ny) - 1.0

# Enforce strict zero mean in the noise
noise -= noise.mean()

# Normalize amplitude so RMS ~ noise_amp
rms_current = np.sqrt(np.mean(noise**2))
noise *= (noise_amp / rms_current)

# Overall domain average is exactly c0
c = c0 + noise
print(f"Initial c average = {c.mean():.6f} (should be ~{c0})")

# ------------------------------------------------
# 4) Free-energy derivative f'(c)
#    f(c) = A * c^2 * (1-c)^2
#    f'(c) = 2 A c (1 - c)(1 - 2 c)
# ------------------------------------------------
def fprime(c):
    return 2.0 * A * c * (1.0 - c) * (1.0 - 2.0*c)

# ------------------------------------------------
# 5) One time step of Cahn-Hilliard (semi-implicit Fourier)
#    PDE: ∂c/∂t = ∇·[ M ∇(f'(c) - kappa ∇² c ) ]
#             = M ∇² [ f'(c) - kappa ∇² c ]
# ------------------------------------------------
def step_ch(c):
    fp = fprime(c)
    c_hat  = np.fft.fft2(c)
    fp_hat = np.fft.fft2(fp)
    
    # Semi-implicit update in Fourier space:
    # c_new(k) = [ c(k) - dt*M*k^2 * fp(k) ] / [ 1 + dt*M*kappa*k^4 ]
    numerator   = c_hat - dt*M*k2*fp_hat
    denominator = 1.0 + dt*M*kappa*k4
    c_hat_new   = numerator / denominator

    c_new = np.fft.ifft2(c_hat_new).real
    return c_new

# ------------------------------------------------
# 6) Visualization setup (Matplotlib Animation)
# ------------------------------------------------
fig, ax = plt.subplots()
im = ax.imshow(c, origin='lower', vmin=0, vmax=1,
               extent=[0, nx*dx, 0, ny*dy], interpolation='bilinear')
fig.colorbar(im, ax=ax, label='Composition')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(f'Cahn-Hilliard (t=0, avg={c0:.3f})')

def update_anim(frame):
    global c
    # Advance 'plot_interval' steps per animation frame
    for _ in range(plot_interval):
        c = step_ch(c)
    im.set_data(c)
    ax.set_title(f'Cahn-Hilliard (t={frame*plot_interval*dt:.2f}, avg={c.mean():.3f})')
    return [im]

anim = FuncAnimation(
    fig, 
    update_anim, 
    frames=int(n_steps/plot_interval),
    interval=100, 
    blit=True
)

plt.close()  # Avoids duplicate static figure in notebooks

# ------------------------------------------------
# 7) Automatic Display Method
#    Try interactive slider via JSHTML; fallback to HTML5
# ------------------------------------------------
try:
    html_anim = HTML(anim.to_jshtml())  # interactive slider
except RuntimeError:
    html_anim = HTML(anim.to_html5_video())  # fallback

display(html_anim)

# ------------------------------------------------
# 8) Add Interactive Buttons for Play, Loop, Stop
# ------------------------------------------------
def run_loop(_):
    """Loop animation indefinitely."""
    anim.event_source.start()

def run_once(_):
    """Run the animation once."""
    anim.event_source.stop()
    anim.frame_seq = anim.new_frame_seq()  # reset
    anim.event_source.start()

def stop_animation(_):
    """Stop animation."""
    anim.event_source.stop()

button_loop = widgets.Button(description="Loop")
button_once = widgets.Button(description="Once")
button_stop = widgets.Button(description="Stop")

button_loop.on_click(run_loop)
button_once.on_click(run_once)
button_stop.on_click(stop_animation)

#display(widgets.HBox([button_loop, button_once, button_stop]))


In [ ]:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Test")
display(button)
